In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

In [2]:
train_df = pd.read_csv("./Data/train.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182080 entries, 0 to 182079
Data columns (total 16 columns):
id                                              182080 non-null object
teacher_id                                      182080 non-null object
teacher_prefix                                  182076 non-null object
school_state                                    182080 non-null object
project_submitted_datetime                      182080 non-null object
project_grade_category                          182080 non-null object
project_subject_categories                      182080 non-null object
project_subject_subcategories                   182080 non-null object
project_title                                   182080 non-null object
project_essay_1                                 182080 non-null object
project_essay_2                                 182080 non-null object
project_essay_3                                 6374 non-null object
project_essay_4                        

In [3]:
test_df = pd.read_csv("./Data/test.csv")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78035 entries, 0 to 78034
Data columns (total 15 columns):
id                                              78035 non-null object
teacher_id                                      78035 non-null object
teacher_prefix                                  78034 non-null object
school_state                                    78035 non-null object
project_submitted_datetime                      78035 non-null object
project_grade_category                          78035 non-null object
project_subject_categories                      78035 non-null object
project_subject_subcategories                   78035 non-null object
project_title                                   78035 non-null object
project_essay_1                                 78035 non-null object
project_essay_2                                 78035 non-null object
project_essay_3                                 2704 non-null object
project_essay_4                                 2704

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#rain_test_df = train_df.append(test_df)
train_test_df = pd.concat([train_df, test_df], axis=0)
train_test_df = train_test_df.reindex(columns=train_df.columns)
train_test_df.info()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


<class 'pandas.core.frame.DataFrame'>
Int64Index: 260115 entries, 0 to 78034
Data columns (total 16 columns):
id                                              260115 non-null object
teacher_id                                      260115 non-null object
teacher_prefix                                  260110 non-null object
school_state                                    260115 non-null object
project_submitted_datetime                      260115 non-null object
project_grade_category                          260115 non-null object
project_subject_categories                      260115 non-null object
project_subject_subcategories                   260115 non-null object
project_title                                   260115 non-null object
project_essay_1                                 260115 non-null object
project_essay_2                                 260115 non-null object
project_essay_3                                 9078 non-null object
project_essay_4                         

In [5]:
train_test_df[182070:182100]

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
182070,p068185,5fcdb7848bf896d1c01e3ff594c73761,Ms.,OK,2017-04-20 10:09:38,Grades 6-8,"Literacy & Language, Math & Science","Literature & Writing, Mathematics",Comfortable Seating Choices for Hamilton!!!,Hamilton Elementary is a Title 1 school that r...,These resources will be used in the classroom ...,NaN,NaN,My students need multiple seating options so t...,0,0.0
182071,p248714,50117fcb5352aaca75760cd39e6a82ae,Ms.,TN,2016-05-28 23:50:29,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",A Rug to Bring Us All Together,My first grade students come to school everyda...,In order to make an my classroom an inviting e...,NaN,NaN,My students need a rug for morning meeting and...,1,1.0
182072,p045565,10b05619a4a0917c2d64bc55b7e1e52d,Mrs.,TX,2016-07-21 16:18:35,Grades PreK-2,"Applied Learning, Literacy & Language","Early Development, Literature & Writing",Kindergarten writing and shelving- ATPE,"I am blessed to work in a rural, but quickly g...",It is my goal to teach my students to be indep...,NaN,NaN,My students need a writing center and shelving...,0,1.0
182073,p078709,e8361d00ab63a2cc420c29cd7cffe26c,Mr.,TX,2016-11-09 13:32:05,Grades PreK-2,Health & Sports,Team Sports,Basket Balling,I have a group of inspiring students who will ...,My students are some of the brightest in the d...,NaN,NaN,My students need basketball equipment that can...,1,0.0
182074,p184627,b59fae3b5cdd41b2b0b87de666f7f22a,Mrs.,CA,2016-08-04 22:30:02,Grades 6-8,"Literacy & Language, Special Needs","Literacy, Special Needs",Books Are The Best,"My students in my class are sixth, seventh and...","I am asking for forty, new, young adult novels...",NaN,NaN,My students need a collection of 40 young adul...,65,1.0
182075,p014188,dd05465e8daad481c9be9e61e147c9cd,Mr.,NM,2016-10-07 16:13:32,Grades 6-8,Math & Science,Mathematics,Math is Fun!!,I currently teach math in a low-income school ...,Students seem to get board with math quickly a...,NaN,NaN,My students need a way to have fun while worki...,1,1.0
182076,p116452,40762708f7589d161ae883dede98a6a1,Mr.,AZ,2017-03-11 23:58:31,Grades 9-12,Music & The Arts,Performing Arts,"\""Sew\"" you think you can design costumes!",My students are predominantly Hispanic and oft...,I was taken back earlier in the year when I br...,NaN,NaN,My students need this serger to make our costu...,0,1.0
182077,p074761,3429d29ecba4a37ac5bd69ded4aab341,Mrs.,AZ,2017-02-09 15:42:43,Grades 6-8,Math & Science,"Applied Sciences, Environmental Science",Sustainable Classroom,I teach science to eighth graders at a suburba...,"With this project, students will be able to ma...",NaN,NaN,"My students need a vinyl machine, button maker...",48,0.0
182078,p136737,497ba7027394d6490d2d2b34af2db5fd,Mrs.,FL,2016-12-28 14:20:07,Grades PreK-2,Literacy & Language,Literacy,Can I go to the library?,I work with a group of wonderful second grade ...,Have you ever went to the library to find a go...,NaN,NaN,My students need 3 shelf organizers to have a ...,29,1.0
182079,p190772,337bc63eccf6f6ee26e727a1f18b80f4,Ms.,TX,2016-05-15 20:25:14,Grades 3-5,Literacy & Language,Literacy,Reader Leaders!,Balanced literacy was a mystery to me until a ...,"At the beginning of the school year, the readi...",My students will be able to listen to books re...,Donations to this project will help my student...,My students need a literacy listening center w...,1,1.0


In [6]:
resource_df = pd.read_csv("./B01.csv")
resource_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260115 entries, 0 to 260114
Data columns (total 5 columns):
id                260115 non-null object
description       260115 non-null object
total_quantity    260115 non-null int64
total_price       260115 non-null float64
cnt_by_id         260115 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 9.9+ MB


In [7]:
train_test_df = pd.merge(train_test_df, resource_df, on='id', how='left')
train_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260115 entries, 0 to 260114
Data columns (total 20 columns):
id                                              260115 non-null object
teacher_id                                      260115 non-null object
teacher_prefix                                  260110 non-null object
school_state                                    260115 non-null object
project_submitted_datetime                      260115 non-null object
project_grade_category                          260115 non-null object
project_subject_categories                      260115 non-null object
project_subject_subcategories                   260115 non-null object
project_title                                   260115 non-null object
project_essay_1                                 260115 non-null object
project_essay_2                                 260115 non-null object
project_essay_3                                 9078 non-null object
project_essay_4                        

In [8]:
train_test_df.head(10)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,description,total_quantity,total_price,cnt_by_id
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1.0,Apple - iPod nano� 16GB MP3 Player (8th Genera...,6,899.94,2
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0.0,Reebok Girls' Fashion Dance Graphic T-Shirt - ...,20,400.00,1
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1.0,3doodler Start Full Edu Bundle,1,469.99,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0.0,BALL PG 4'' POLY SET OF 6 COLORS BALL PLAYGROU...,5,684.47,5
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1.0,Crown Berkey Water Filter With 2 Black and 2 P...,2,711.00,1
5,p063374,403c6783e9286e51ab318fba40f8d729,Mrs.,DE,2016-11-05 10:01:51,Grades PreK-2,"Applied Learning, Literacy & Language","Character Education, Literature & Writing",Need to Reach Our Virtual Mentors!!!,My kids tell me each day that they want to mak...,I started a program called Telementoring in ho...,NaN,NaN,My students need tablets in order to communic...,0,1.0,Amazon - Fire Kids Edition - 7'- Tablet - 16GB...,7,727.36,2
6,p103285,4e156c5fb3eea2531601c8736f3751a7,Mrs.,MO,2016-08-31 00:30:43,Grades PreK-2,Health & Sports,Health & Wellness,Active Kindergartners,Kindergarten is the new first grade. My studen...,With balance discs and stools as flexible seat...,NaN,NaN,My students need stability stools and inflatab...,1,1.0,"Hokki Stool - 15"" Black Therapist's Choice® In...",6,414.02,2
7,p181781,c71f2ef13b4bc91afac61ca8fd4c0f9f,Mrs.,SC,2016-08-03 13:26:01,Grades PreK-2,"Applied Learning, Literacy & Language","Early Development, Literature & Writing",Fabulous Firsties-Wiggling to Learn!,First graders are fantastic! They are excited ...,First graders love learning! We need 6 wiggle-...,NaN,NaN,My students need wiggle stools to allow them t...,0,1.0,"ECR4Kids ACE Active Sitting Stool, 15"", Black",6,414.78,1
8,p114989,b580c11b1497a0a67317763b7f03eb27,Ms.,IN,2016-09-13 22:35:57,Grades 6-8,Math & Science,Mathematics,Wobble Chairs Help Fidgety Kids Focus,My seventh graders dream big. They can't wait ...,I have used alternative seating in my classroo...,NaN,NaN,My students need seating that allows the most ...,13,1.0,"Kore Patented WOBBLE Chair, Made in the USA, A...",4,319.80,1
9,p191410,2071fb0af994f8f16e7c6ed0f35062a1,Mrs.,IL,2016-09-24 18:38:59,Grades PreK-2,Literacy 

In [9]:
train_test_df.to_csv("B02.csv", index=False, quoting=csv.QUOTE_NONNUMERIC , quotechar='"')

NameError: name 'csv' is not defined

In [ ]:
dummy_df = pd.read_csv("B02.csv")
dummy_df.info()

In [ ]:
dummy_df.head(10)